In [ ]:
import geopandas
import momepy
import momepy.coins
import numpy as np
import sklearn.cluster
import shapely

# Corridor Segments

Load the street network (edges only), the waterway, and the corridor: 

In [ ]:
city_name = "Bucharest"
river_name = "Dâmbovița"

In [ ]:
edges = geopandas.read_file(f"../../data/generated/street_network_edges_{city_name}.gpkg")
waterway = geopandas.read_file(f"../../data/generated/waterway_{river_name}.gpkg")
corridor = geopandas.read_file(f"../../data/generated/corridor_{river_name}.gpkg")


Extract the relevant geometries:

In [ ]:
waterway_geometry = waterway.iloc[0].geometry
corridor_geometry = corridor.iloc[0].geometry
edge1_geometry = corridor.iloc[1].geometry
edge2_geometry = corridor.iloc[2].geometry

## Continuity analysis (modified `momepy`)

In [ ]:
def _cross_check_links(unique, angle_pairs, angle_threshold):
    """
    Modified version of the momepy function that identifies "best"
    links between segments, dropping the reciprocity requirement.
    """
    for edge in range(0, len(unique)):
        best_p1 = unique[edge][4][0]
        best_p2 = unique[edge][5][0]

        if (
            isinstance(best_p1, int)
            # and edge in [unique[best_p1][4][0], unique[best_p1][5][0]]
            and angle_pairs["%d_%d" % (edge, best_p1)] > angle_threshold
        ):
            unique[edge][6] = best_p1
        else:
            unique[edge][6] = "line_break"

        if (
            isinstance(best_p2, int)
            # and edge in [unique[best_p2][4][0], unique[best_p2][5][0]]
            and angle_pairs["%d_%d" % (edge, best_p2)] > angle_threshold
        ):
            unique[edge][7] = best_p2
        else:
            unique[edge][7] = "line_break"

In [ ]:
def _merge_lines_loop(n, unique_dict, bound_geometry):
    """
    Modified version of the momepy function that merges the line
    segments:
    - crossing a provided boundary geometry is added as stopping condition;
    - when assigning the best continuation segment on a vertex, we force the
      following iteration to continue on the other vertex of the segment.
      This is probably needed as a consequence of having dropped the reciprocity
      requirement in assigning the best links.
    """
    outlist = set()
    current_edge1 = n
    vertex = None

    outlist.add(current_edge1)
    while True:
        p1, p2 = [tuple(i) for i in unique_dict[current_edge1][0]]
        edge = shapely.LineString([p1, p2])
        if not bound_geometry.contains(edge):
            break
        if (
            isinstance(unique_dict[current_edge1][6], int)
            and unique_dict[current_edge1][6] not in outlist
            and ((vertex is None) or (vertex == p2))
        ):
            current_edge1 = unique_dict[current_edge1][6]
            outlist.add(current_edge1)
            vertex = tuple(p1)
        elif (
            isinstance(unique_dict[current_edge1][7], int)
            and unique_dict[current_edge1][7] not in outlist
            and ((vertex is None) or (vertex == p1))
        ):
            current_edge1 = unique_dict[current_edge1][7]
            outlist.add(current_edge1)
            vertex = tuple(p2)
        else:
            break

    current_edge1 = n
    vertex = None
    while True:
        p1, p2 = [tuple(i) for i in unique_dict[current_edge1][0]]
        edge = shapely.LineString([p1, p2])
        if not bound_geometry.contains(edge):
            break
        if (
            isinstance(unique_dict[current_edge1][7], int)
            and unique_dict[current_edge1][7] not in outlist
            and ((vertex is None) or (vertex == p1))
        ):
            current_edge1 = unique_dict[current_edge1][7]
            outlist.add(current_edge1)
            vertex = tuple(p2)
        elif (
            isinstance(unique_dict[current_edge1][6], int)
            and unique_dict[current_edge1][6] not in outlist
            and ((vertex is None) or (vertex == p2))
        ):
            current_edge1 = unique_dict[current_edge1][6]
            outlist.add(current_edge1)
            vertex = tuple(p1)
        else:
            break

    outlist = list(outlist)
    outlist.sort()
    return outlist


In [ ]:
edges_simplified = momepy.roundabout_simplification(edges)

In [ ]:
# show effect of roundabout simplification
m = edges.explore(color="red")
m = edges_simplified.explore(m=m, color="black")
m

In [ ]:
# make continuity analysis using COINS
ANGLE_THRESHOLD = 120.
coins = momepy.coins.COINS(edges, angle_threshold=ANGLE_THRESHOLD, flow_mode=True)
# get GDF of all line segments
premerge = coins._create_gdf_premerge()
# find the segments intersecting the waterway
mask = premerge.intersects(waterway_geometry)
crossings = premerge[mask]

In [ ]:
# find the crossing points as intersections between the waterway and the street network
crossing_points = crossings.intersection(waterway_geometry) # this should now be "point" geometries

In [ ]:
# group the crossing points in clusters
xy = np.column_stack([crossing_points.x, crossing_points.y])
dbscan = sklearn.cluster.DBSCAN(eps=100, min_samples=1)
dbscan.fit(xy)

In [ ]:
# modified assignment of best continuation edges (with threshold)
_cross_check_links(coins.unique, coins.angle_pairs, ANGLE_THRESHOLD)

In [ ]:
# grow strokes fromm the crossings
geoms = []
for idx in crossings.index:
    indices = _merge_lines_loop(idx, coins.unique, corridor_geometry)
    geoms.append(premerge.loc[indices].unary_union)
strokes = geopandas.GeoDataFrame(geometry=geoms, crs=crossings.crs)
strokes["cluster"] = dbscan.labels_

In [ ]:
# find the ones that intersect both edges of the corridor
mask = strokes.intersects(edge1_geometry) & strokes.intersects(edge2_geometry)
filtered = strokes[mask]

In [ ]:
# for each cluster, select the shortest
filtered = filtered.assign(length=filtered.length)
idx = filtered.groupby("cluster").agg({"length": "idxmin"})
shortest = filtered.loc[idx["length"].values]

In [ ]:
m = corridor.explore()
m = shortest.explore(m=m, color="red")
m = crossings.explore(m=m, color="green")
m